In [1]:
import numpy as np
import pandas as pd

In [2]:
from __future__ import print_function

In [3]:
all_tracks = pd.read_csv('all_tracks.csv', index_col = 12)
all_tracks.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature,popularity,main_artist,playlist
id,,,,,,,,,,,,,,,,,,,,
7ltsfuHdqTZ5LwPpDy1q0v,0.786,0.579,2,-14.913,1,0.1420,0.1860,0.000000,0.1180,0.481,90.012,audio_features,spotify:track:7ltsfuHdqTZ5LwPpDy1q0v,https://api.spotify.com/v1/tracks/7ltsfuHdqTZ5...,https://api.spotify.com/v1/audio-analysis/7lts...,416613,4,45,Sukhwinder Singh,Hindi
0OXQa6GOzxeYWFopEmASw9,0.805,0.670,2,-13.187,1,0.0601,0.0493,0.000050,0.2030,0.916,122.992,audio_features,spotify:track:0OXQa6GOzxeYWFopEmASw9,https://api.spotify.com/v1/tracks/0OXQa6GOzxeY...,https://api.spotify.com/v1/audio-analysis/0OXQ...,425413,4,18,Shankar Mahadevan,Hindi
6B58TJqjGLQuuZBhQ3UnEA,0.767,0.503,9,-13.424,1,0.0828,0.1200,0.000096,0.1580,0.483,141.996,audio_features,spotify:track:6B58TJqjGLQuuZBhQ3UnEA,https://api.spotify.com/v1/tracks/6B58TJqjGLQu...,https://api.spotify.com/v1/audio-analysis/6B58...,309107,4,35,Lata Mangeshkar,Hindi
0IYpKZhCeoXB85BslBg3q5,0.646,0.908,11,-4.680,0,0.0988,0.4190,0.000002,0.3810,0.881,104.238,audio_features,spotify:track:0IYpKZhCeoXB85BslBg3q5,https://api.spotify.com/v1/tracks/0IYpKZhCeoXB...,https://api.spotify.com/v1/audio-analysis/0IYp...,327227,4,45,Jatin-Lalit,Hindi
7CcAKQatXMmlr7wi6BaQsI,0.493,0.947,1,-4.188,0,0.2060,0.0815,0.009370,0.0552,0.568,127.986,audio_features,spotify:track:7CcAKQatXMmlr7wi6BaQsI,https://api.spotify.com/v1/tracks/7CcAKQatXMml...,https://api.spotify.com/v1/audio-analysis/7CcA...,297927,4,40,Aditya Narayan,Hindi


In [4]:
all_tracks = all_tracks.drop(['type','uri','track_href', 'analysis_url'], axis = 1)
all_tracks.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,popularity,main_artist,playlist
id,,,,,,,,,,,,,,,,
7ltsfuHdqTZ5LwPpDy1q0v,0.786,0.579,2,-14.913,1,0.1420,0.1860,0.000000,0.1180,0.481,90.012,416613,4,45,Sukhwinder Singh,Hindi
0OXQa6GOzxeYWFopEmASw9,0.805,0.670,2,-13.187,1,0.0601,0.0493,0.000050,0.2030,0.916,122.992,425413,4,18,Shankar Mahadevan,Hindi
6B58TJqjGLQuuZBhQ3UnEA,0.767,0.503,9,-13.424,1,0.0828,0.1200,0.000096,0.1580,0.483,141.996,309107,4,35,Lata Mangeshkar,Hindi
0IYpKZhCeoXB85BslBg3q5,0.646,0.908,11,-4.680,0,0.0988,0.4190,0.000002,0.3810,0.881,104.238,327227,4,45,Jatin-Lalit,Hindi
7CcAKQatXMmlr7wi6BaQsI,0.493,0.947,1,-4.188,0,0.2060,0.0815,0.009370,0.0552,0.568,127.986,297927,4,40,Aditya Narayan,Hindi


In [5]:
msk = np.random.randn(len(all_tracks)) < .8
training = all_tracks[msk]
testing= all_tracks[~msk]

In [6]:
def list_of_lister(df):
    row_list = []
    for index, row in df.iterrows():
        row_list.append(list(row))
    return row_list
all_tracks_input = list_of_lister(all_tracks)
training_data = list_of_lister(training)
testing_data = list_of_lister(testing)

In [7]:
# column labels
column_labels = list(all_tracks.columns)
print(column_labels)

['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature', 'popularity', 'main_artist', 'playlist']


In [8]:
# Finding the number of unique playlists within the dataset
def unique_vals(rows, col):
    return set(row[col] for row in rows)
unique_vals(all_tracks_input, -1)

{'Frictionless', 'Hindi', "I'm a god", 'Peak Trap'}

In [9]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [10]:
class_counts(all_tracks_input)

{'Frictionless': 100, 'Hindi': 48, "I'm a god": 89, 'Peak Trap': 35}

In [11]:
def  is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [12]:
class  Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            column_labels[self.column], condition, str(self.value))

In [13]:
q = Question(-1, 'Frictionless')

In [14]:
example = all_tracks.iloc[1]

In [15]:
q.match(example)

False

In [16]:
# This function partitions a dataset and checks if it matches the question
# If it matches the question it is added to true_rows
# Else it is added to false_rows
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [17]:
#######
# Demo:
# Let's partition the training data based on whether or not the song is in the playlist Frictionless.
true_rows, false_rows = partition(all_tracks_input, Question(-1, "Frictionless"))

In [18]:
# Calculating gini coefficient of rows
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [19]:
gini(true_rows)

0.0

In [20]:
gini(false_rows)

0.6129664683612763

In [21]:
gini(all_tracks_input)

0.7100724480968857

In [22]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [23]:
#######
# Demo:
# Calculate the uncertainy of our training data.
current_uncertainty = gini(training_data)
current_uncertainty

0.713239626294083

In [24]:
# How much information do we gain by partioning on Popularity = 50?
true_rows, false_rows = partition(training_data, Question(-3, 50))
info_gain(true_rows, false_rows, current_uncertainty)

0.13768318586030523

In [25]:
def  find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    index_features = [0,1,3,5,6,13]  # index of the features in each list

    for num in index_features:  # for each feature

        values = set([row[num] for row in rows])  # unique values for each feature

        for val in values:  # for each value

            question = Question(num, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [26]:
#######
# Demo:
# Find the best question to ask first for our toy dataset.
best_gain, best_question = find_best_split(training_data)
best_question
# FYI: is color == Red is just as good. See the note in the code above
# where I used '>='.
#######

Is popularity >= 50?

In [27]:
# This class is used to represent leaf nodes in our decision tree, leaf nodes classify data. The class Leaf holds
# a dictionary of target values and the number of times it appears in the rows that have made it to this leaf node
class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [28]:
# Next we will initialize a class called Decision_Node, in decision trees, a decision node asks a question, this class holds
# reference to a question and to the child_nodes
class Decision_Node:
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [29]:
# Now we have a function called build_tree, which takes in the data as an input and builds a tree recursively
def build_tree(rows):
    gain, question = find_best_split(rows)
    
    if gain == 0:
        return Leaf(rows)
    
    true_rows, false_rows = partition(rows, question)
    
    true_branch = build_tree(true_rows)
    
    false_branch = build_tree(false_rows)
    
    return Decision_Node(question, true_branch, false_branch)

In [30]:
# Next we have a function called print_tree which basically allows us to print our tree in a pretty format
def print_tree(node, spacing = ""):
    if isinstance(node, Leaf):
        print(spacing + "Predict:", node.predictions)
        return
    print(spacing + str(node.question))
    
    print(spacing + "--> True")
    print_tree(node.true_branch, spacing + "    ")
    
    print(spacing + "--> False")
    print_tree(node.false_branch, spacing + "    ")
    

In [31]:
my_tree = build_tree(training_data)

In [32]:
print_tree(my_tree)

Is popularity >= 50?
--> True
    Is acousticness >= 0.0957?
    --> True
        Is speechiness >= 0.129?
        --> True
            Is danceability >= 0.828?
            --> True
                Predict: {"I'm a god": 12}
            --> False
                Is loudness >= -4.104?
                --> True
                    Predict: {"I'm a god": 4}
                --> False
                    Is loudness >= -8.571?
                    --> True
                        Is acousticness >= 0.5429999999999999?
                        --> True
                            Is popularity >= 73?
                            --> True
                                Predict: {'Frictionless': 1}
                            --> False
                                Is speechiness >= 0.213?
                                --> True
                                    Predict: {"I'm a god": 2}
                                --> False
                                    Predict: {'Frictionless':

In [33]:
# This next function is called classify, which once again is a recursive function, it takes in a given row of data in
# as an input and node, which is a decision node output as another input
def classify(row, node):
    if isinstance(node, Leaf):
        return node.predictions
    
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [34]:
#######
# Demo:
# The tree predicts the 1st row of our
# training data is an apple with confidence 1.
classify(training_data[0], my_tree)
#######

{'Hindi': 9}

In [35]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [36]:
#######
# Demo:
# Printing that a bit nicer
print_leaf(classify(training_data[0], my_tree))
#######

{'Hindi': '100%'}

In [37]:
#######
# Demo:
# On the second example, the confidence is lower
print_leaf(classify(training_data[96], my_tree))
#######

{'Frictionless': '100%'}

In [38]:
row_list = []
item_list = []
from sklearn.metrics import accuracy_score
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))
    row_list.append(row[-1])
    for item in print_leaf(classify(row,my_tree)).keys():
        item_list.append(item)
print(accuracy_score(row_list, item_list))

Actual: Hindi. Predicted: {'Peak Trap': '100%'}
Actual: Hindi. Predicted: {'Hindi': '100%'}
Actual: Hindi. Predicted: {'Hindi': '100%'}
Actual: Hindi. Predicted: {'Hindi': '100%'}
Actual: Hindi. Predicted: {'Hindi': '100%'}
Actual: Hindi. Predicted: {'Hindi': '100%'}
Actual: Peak Trap. Predicted: {'Peak Trap': '100%'}
Actual: Peak Trap. Predicted: {"I'm a god": '100%'}
Actual: Peak Trap. Predicted: {"I'm a god": '100%'}
Actual: Peak Trap. Predicted: {'Peak Trap': '100%'}
Actual: Peak Trap. Predicted: {"I'm a god": '100%'}
Actual: Peak Trap. Predicted: {'Hindi': '100%'}
Actual: Peak Trap. Predicted: {'Frictionless': '100%'}
Actual: Frictionless. Predicted: {'Frictionless': '100%'}
Actual: Frictionless. Predicted: {'Frictionless': '100%'}
Actual: Frictionless. Predicted: {'Peak Trap': '100%'}
Actual: Frictionless. Predicted: {'Frictionless': '100%'}
Actual: Frictionless. Predicted: {'Frictionless': '100%'}
Actual: Frictionless. Predicted: {'Frictionless': '100%'}
Actual: Frictionless. Pr

In [39]:
len(testing_data)

54